In [22]:
import numpy as np
import pandas as pd
import cv2
import os
import math

In [5]:
df1 = pd.read_excel("../Dataset Used/Blood Report_without patient.xlsx",sheet_name="left_eye")
df1.head()

,1. Take a photo of Left eye,Blood Sample Id,Unnamed: 2,Unnamed: 3,Annotated_Data,Avaiable ID,Hb Value
0,1709617490220.jpg,202403051,NaN,NaN,1709617490220.jpg,202403051.0,11.5
1,1709619661748.jpg,202403058,NaN,NaN,1709617782204.jpg,202403052.0,11.6
2,1709621461793.jpg,2024030512,NaN,NaN,1709618916134.jpg,202403056.0,11.1
3,1709624290874.jpg,2024030518,NaN,NaN,1709619661748.jpg,202403058.0,12.1
4,1709625767525.jpg,2024030522,NaN,NaN,1709620146694.jpg,202403059.0,12.2


In [6]:
df2 = pd.read_excel("../Dataset Used/Updated Healthworker Form Part 1_without patient identifier.xlsx") 
df2.head()

,start,end,today,deviceid,username,Do you give consent?,Patient Id,Blood Sample Id,2. Enter age in completed years,4. Patient Sex,...,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,__version__,_tags,_index
0,2024-03-05T11:01:43.141+05:30,2024-03-05T12:36:06.048+05:30,2024-03-05,collect:wVnGkN86bc6HA9Fi,shila,Yes,1,202403051,63,Female,...,116,e7525154-4a03-4818-b76a-85f5d5657f73,2024-03-05T17:57:45,NaN,NaN,submitted_via_web,shila,v9nwsZRTukYiPhwTrkJqPu,NaN,1
1,2024-03-05T11:39:13.925+05:30,2024-03-05T11:55:12.666+05:30,2024-03-05,collect:wVnGkN86bc6HA9Fi,shila,Yes,8,202403058,42,Female,...,117,a407f9d7-8643-4a35-a614-3b83184b83e0,2024-03-05T17:57:46,NaN,NaN,submitted_via_web,shila,v9nwsZRTukYiPhwTrkJqPu,NaN,2
2,2024-03-05T12:07:50.954+05:30,2024-03-05T12:26:37.006+05:30,2024-03-05,collect:wVnGkN86bc6HA9Fi,shila,Yes,12,2024030512,65,Male,...,119,3f1f095e-d9d1-4a6c-bf3e-616632cf64ac,2024-03-05T17:57:47,NaN,NaN,submitted_via_web,shila,v9nwsZRTukYiPhwTrkJqPu,NaN,3
3,2024-03-05T12:50:11.280+05:30,2024-03-05T13:13:53.977+05:30,2024-03-05,collect:wVnGkN86bc6HA9Fi,shila,Yes,18,2024030518,75,Female,...,121,7be32b46-35d2-4923-b256-ef11b633fd05,2024-03-05T17:57:52,NaN,NaN,submitted_via_web,shila,v9nwsZRTukYiPhwTrkJqPu,NaN,4
4,2024-03-05T13:21:52.005+05:30,2024-03-05T13:40:16.946+05:30,2024-03-05,collect:wVnGkN86bc6HA9Fi,shila,Yes,22,2024030522,38,Female,...,122,b7d5c35e-cba8-46ff-9e2e-9d82ec8eedc5,2024-03-05T17:57:54,NaN,NaN,submitted_via_web,shila,v9nwsZRTukYiPhwTrkJqPu,NaN,5


In [7]:
df1_filtered = df1[['Annotated_Data', 'Blood Sample Id', 'Hb Value']].rename(
    columns={'Annotated_Data': 'Image Name', 'Hb Value': 'Hb Level'}
)

In [9]:
df1_filtered.head()

,Image Name,Blood Sample Id,Hb Level
0,1709617490220.jpg,202403051,11.5
1,1709617782204.jpg,202403058,11.6
2,1709618916134.jpg,2024030512,11.1
3,1709619661748.jpg,2024030518,12.1
4,1709620146694.jpg,2024030522,12.2


In [10]:
df2_filtered = df2[['Blood Sample Id', '4. Patient Sex', '2. Enter age in completed years']].rename(
    columns={'4. Patient Sex': 'Gender', '2. Enter age in completed years': 'Age'}
)

In [11]:
df2_filtered.head()

,Blood Sample Id,Gender,Age
0,202403051,Female,63
1,202403058,Female,42
2,2024030512,Male,65
3,2024030518,Female,75
4,2024030522,Female,38


In [12]:
merged_df = pd.merge(df1_filtered, df2_filtered, on='Blood Sample Id', how='inner')

In [13]:
merged_df.head()

,Image Name,Blood Sample Id,Hb Level,Gender,Age
0,1709617490220.jpg,202403051,11.5,Female,63
1,1709617782204.jpg,202403058,11.6,Female,42
2,1709618916134.jpg,2024030512,11.1,Male,65
3,1709619661748.jpg,2024030518,12.1,Female,75
4,1709620146694.jpg,2024030522,12.2,Female,38


In [14]:
final_df = merged_df[['Image Name', 'Gender', 'Age', 'Hb Level']]
final_df.head()

,Image Name,Gender,Age,Hb Level
0,1709617490220.jpg,Female,63,11.5
1,1709617782204.jpg,Female,42,11.6
2,1709618916134.jpg,Male,65,11.1
3,1709619661748.jpg,Female,75,12.1
4,1709620146694.jpg,Female,38,12.2


In [15]:
final_df['Gender'] = final_df['Gender'].map({'Male':0, 'Female':1})
final_df.head()

/tmp/ipykernel_87658/3814951379.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Gender'] = final_df['Gender'].map({'Male':0, 'Female':1})


,Image Name,Gender,Age,Hb Level
0,1709617490220.jpg,1,63,11.5
1,1709617782204.jpg,1,42,11.6
2,1709618916134.jpg,0,65,11.1
3,1709619661748.jpg,1,75,12.1
4,1709620146694.jpg,1,38,12.2


In [16]:
eye_mask_folder = "../Produced Images/Eye Masked"

In [17]:
def pooling(image, pool_size, code, padding):
    # Simple pooling implementation for demonstration
    pad_size = pool_size[0] // 2
    image_padded = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size)), mode='constant')
    rows, cols = image.shape
    output = np.zeros_like(image, dtype=np.float32)

    for i in range(rows):
        for j in range(cols):
            region = image_padded[i:i + pool_size[0], j:j + pool_size[1]]
            if code == 'min':
                output[i, j] = region.min()
            elif code == 'max':
                output[i, j] = region.max()
            elif code == 'mean':
                output[i, j] = region.mean()
            elif code == 'std':
                output[i, j] = region.std()
    return output

In [19]:
def feature(image_path):
    img = cv2.imread(image_path)
    # Resize the image
    img = cv2.resize(img, (250, 250))

    b, g, r = cv2.split(img)
    i = 0
    meanb, meang, meanr = 0, 0, 0
    for value in b.flatten():
        if value > 0:
            meanb += value
            i += 1
    mean_b = meanb / i
    for value in g.flatten():
        meang += value
    mean_g = meang / i
    for value in r.flatten():
        meanr += value
    mean_r = meanr / i
    mean_rg = (mean_r - mean_g) / i
    EI = np.log10(mean_r / mean_g)

    # HSV SPACE
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    h = h / h.max()
    nH = np.count_nonzero(h > 0.95)
    HHR = nH / i

    # GRAY SPACE
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    B = 0
    for value in gray.flatten():
        B += value
    B = B / i

    # ENTROPY in gray space
    eq = cv2.equalizeHist(gray)
    unique, counts = np.unique(eq, return_counts=True)
    total_counts = counts[21:240].sum()
    Ent = np.sum([-i * (i / total_counts) * math.log((i / total_counts), 2) for i in counts[21:240]])

    # Calculating the 'G' features
    Ixy = gray.astype(np.float32)
    min_Ixy = pooling(Ixy, pool_size=(3, 3), code='min', padding=1)
    max_Ixy = pooling(Ixy, pool_size=(3, 3), code='max', padding=1)
    mean_Ixy = pooling(Ixy, pool_size=(3, 3), code='mean', padding=1)
    std_Ixy = pooling(Ixy, pool_size=(3, 3), code='std', padding=1)

    g1 = Ixy - min_Ixy
    g2 = max_Ixy - Ixy
    g3 = Ixy - mean_Ixy
    g4 = std_Ixy
    g5 = Ixy

    G1 = g1.sum() / g1.size
    G2 = g2.sum() / g2.size
    G3 = g3.sum() / g3.size
    G4 = g4.sum() / g4.size
    G5 = g5.sum() / g5.size

    return [mean_r, mean_g, mean_b, mean_rg, EI, HHR, Ent, B, G1, G2, G3, G4, G5]


In [24]:
features = []
for image_name in final_df['Image Name']:
    if pd.notna(image_name):  # Check if the image name is not NaN
        image_name = str(image_name)  # Ensure the image name is a string
        image_path = os.path.join(eye_mask_folder, image_name)
        if os.path.exists(image_path):  # Check if the image file exists
            features.append(feature(image_path))
        else:
            features.append([np.nan] * 13)  # Handle missing images
    else:
        features.append([np.nan] * 13)  # Handle missing or NaN image names

# Add features to the DataFrame
feature_columns = [
    'Mean_R', 'Mean_G', 'Mean_B', 'Mean_RG', 'EI', 'HHR', 'Ent', 'B',
    'G1', 'G2', 'G3', 'G4', 'G5'
]
features_df = pd.DataFrame(features, columns=feature_columns)
final_df = pd.concat([final_df, features_df], axis=1)



In [25]:
final_df.head()

,Image Name,Gender,Age,Hb Level,Mean_R,Mean_G,Mean_B,Mean_RG,EI,HHR,Ent,B,G1,G2,G3,G4,G5
0,1709617490220.jpg,1,63,11.5,138.415323,58.290323,54.233871,0.323085,0.375588,0.020161,27.193427,81.790323,0.165280,0.201728,0.000000e+00,0.141543,0.324544
1,1709617782204.jpg,1,42,11.6,116.540650,53.727642,50.264228,0.255337,0.336280,0.036585,25.923115,72.117886,0.155056,0.182816,-4.882812e-10,0.128795,0.283856
2,1709618916134.jpg,0,65,11.1,122.335526,54.789474,58.256579,0.444382,0.348855,0.605263,14.925607,75.434211,0.130112,0.161728,9.384155e-10,0.112978,0.183456
3,1709619661748.jpg,1,75,12.1,119.631818,48.686364,47.845455,0.322479,0.390439,0.054545,23.042254,69.804545,0.134768,0.169696,0.000000e+00,0.117356,0.245712
4,1709620146694.jpg,1,38,12.2,137.275862,70.409195,64.636782,0.153716,0.289965,0.006897,46.450271,89.754023,0.237408,0.268720,4.882812e-10,0.193553,0.624688


In [28]:
final_df

,Image Name,Gender,Age,Hb Level,Mean_R,Mean_G,Mean_B,Mean_RG,EI,HHR,Ent,B,G1,G2,G3,G4,G5
0,1709617490220.jpg,1,63,11.5,138.415323,58.290323,54.233871,0.323085,0.375588,0.020161,27.193427,81.790323,0.165280,0.201728,0.000000e+00,0.141543,0.324544
1,1709617782204.jpg,1,42,11.6,116.540650,53.727642,50.264228,0.255337,0.336280,0.036585,25.923115,72.117886,0.155056,0.182816,-4.882812e-10,0.128795,0.283856
2,1709618916134.jpg,0,65,11.1,122.335526,54.789474,58.256579,0.444382,0.348855,0.605263,14.925607,75.434211,0.130112,0.161728,9.384155e-10,0.112978,0.183456
3,1709619661748.jpg,1,75,12.1,119.631818,48.686364,47.845455,0.322479,0.390439,0.054545,23.042254,69.804545,0.134768,0.169696,0.000000e+00,0.117356,0.245712
4,1709620146694.jpg,1,38,12.2,137.275862,70.409195,64.636782,0.153716,0.289965,0.006897,46.450271,89.754023,0.237408,0.268720,4.882812e-10,0.193553,0.624688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,NaN,1,62,13.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,NaN,1,29,12.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435,NaN,1,21,13.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
436,NaN,1,75,9.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
final_df  = final_df.dropna(subset=['Image Name'])

In [32]:
final_df

,Image Name,Gender,Age,Hb Level,Mean_R,Mean_G,Mean_B,Mean_RG,EI,HHR,Ent,B,G1,G2,G3,G4,G5
0,1709617490220.jpg,1,63,11.5,138.415323,58.290323,54.233871,0.323085,0.375588,0.020161,27.193427,81.790323,0.165280,0.201728,0.000000e+00,0.141543,0.324544
1,1709617782204.jpg,1,42,11.6,116.540650,53.727642,50.264228,0.255337,0.336280,0.036585,25.923115,72.117886,0.155056,0.182816,-4.882812e-10,0.128795,0.283856
2,1709618916134.jpg,0,65,11.1,122.335526,54.789474,58.256579,0.444382,0.348855,0.605263,14.925607,75.434211,0.130112,0.161728,9.384155e-10,0.112978,0.183456
3,1709619661748.jpg,1,75,12.1,119.631818,48.686364,47.845455,0.322479,0.390439,0.054545,23.042254,69.804545,0.134768,0.169696,0.000000e+00,0.117356,0.245712
4,1709620146694.jpg,1,38,12.2,137.275862,70.409195,64.636782,0.153716,0.289965,0.006897,46.450271,89.754023,0.237408,0.268720,4.882812e-10,0.193553,0.624688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,1710070755134.jpg,0,27,9.6,70.830116,36.073359,40.081081,0.134196,0.293031,0.795367,39.027177,46.942085,0.096208,0.136752,6.103516e-11,0.081366,0.194528
412,1710071452062.jpg,1,27,11.9,143.253247,77.870130,89.915584,0.141522,0.264734,0.790043,35.322736,98.794372,0.289088,0.373360,0.000000e+00,0.235112,0.730288
413,1710071755958.jpg,1,75,9.1,68.654930,30.769953,38.389671,0.088932,0.348545,0.861502,66.605470,42.988263,0.119616,0.143472,-6.103516e-10,0.094703,0.293008
414,1710072078629.jpg,1,29,13.9,104.073684,32.563158,39.313158,0.188186,0.504614,0.805263,36.353271,54.721053,0.152784,0.207296,-2.441406e-10,0.128507,0.332704


In [36]:
final_df.isna().sum()

Image Name    0
Gender        0
Age           0
Hb Level      0
Mean_R        0
Mean_G        0
Mean_B        0
Mean_RG       0
EI            0
HHR           0
Ent           0
B             0
G1            0
G2            0
G3            0
G4            0
G5            0
dtype: int64

In [37]:
final_df.to_excel("Final_Image_Features_Enhanced.xlsx", index=False)
